<h1>IBM Data Science Professional Certificate Capstone</h1>
<h3>Opening a Portuguese Restaurant in Toronto</h3>

<h4> 1 - Import Libraries</h4>

In [1]:
import numpy as np
import pandas as pd
import json
import requests
from bs4 import BeautifulSoup

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

<h4> 2 - Scrape Wikipedia data using BeautifulSoup</h4>

In [2]:
# Get Wikipedia Page text
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

# Parse using Beautifulsoup
soup = BeautifulSoup(data, 'html.parser')
#print(soup.prettify())

In [3]:
scrapeData = []

# Search complete table to get data
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        scrapeData.append([cells[0].text,cells[1].text,cells[2].text])
        #neighborhoodList.append(cells[2].text.rstrip('\n')) # avoid new lines in neighborhood cell
        
scrapeData[0:5]

[['M1A', 'Not assigned', 'Not assigned\n'],
 ['M2A', 'Not assigned', 'Not assigned\n'],
 ['M3A', 'North York', 'Parkwoods\n'],
 ['M4A', 'North York', 'Victoria Village\n'],
 ['M5A', 'Downtown Toronto', 'Harbourfront\n']]

<h4> 3 - Create Data Frame</h4>

In [4]:
# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [5]:
# Add items from scraped data into dataframe
for data in scrapeData:
    neighborhoods = neighborhoods.append({'PostalCode': data[0].rstrip(), #removing trailing spaces and newlines
                                          'Borough': data[1].rstrip(),
                                          'Neighborhood': data[2].rstrip()},
                                          ignore_index=True)

In [6]:
neighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [7]:
# Remove Not assigned
neighborhoods = neighborhoods[neighborhoods.Borough != "Not assigned"].reset_index(drop=True)
neighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [8]:
# Merge neighborhoods in same postal code area
neighborhoods = neighborhoods.groupby(['PostalCode','Borough'])['Neighborhood'].apply(lambda x: ','.join(x)).reset_index()
neighborhoods[neighborhoods.PostalCode == 'M5A']

,PostalCode,Borough,Neighborhood
53,M5A,Downtown Toronto,"Harbourfront,Regent Park"


In [9]:
# Copy Borough into Neighborhood when not assigned
def set_Neighborhood(row):
    if(row['Neighborhood'] == "Not assigned"):
        return row['Borough']
    else:
        return row['Neighborhood']

neighborhoods['Neighborhood'] = neighborhoods.apply(lambda row: set_Neighborhood(row), axis=1)
neighborhoods[neighborhoods.PostalCode == 'M7A'] #Queens Park

,PostalCode,Borough,Neighborhood
85,M7A,Queen's Park,Queen's Park


In [10]:
#print shape of cleaned dataframe
neighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


<h4> 4 - Add Geo Data</h4>

In [11]:
# Read CSV
geoCoords = pd.read_csv("Geospatial_Coordinates.csv")
geoCoords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
# rename the column "PostalCode"
geoCoords.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
geoCoords.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
# Merge data using PostalCode as key
neighborhoods = neighborhoods.merge(geoCoords, on="PostalCode", how="left")
neighborhoods[neighborhoods.PostalCode == 'M5G'] # Downtown Toronto

,PostalCode,Borough,Neighborhood,Latitude,Longitude
57,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


<h4> 5 - Explore and cluster neighborhoods in Toronto</h4>

In [14]:
# create map of Toronto using latitude and longitude values
latitude = 43.6529
longitude = -79.3849
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

<b> Define Foursquare Credentials and get Portuguese venues data<b>

In [15]:
CLIENT_ID = 'AVHL5JQPWFNORX24DUOCV325E431AWJLTBPFJUDWWFSG3CPC' # your Foursquare ID
CLIENT_SECRET = 'DYPYXVYCV5DBLOVTOKWZ42LEFFQ1TRZHK23UGG0TYJWDYKGU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AVHL5JQPWFNORX24DUOCV325E431AWJLTBPFJUDWWFSG3CPC
CLIENT_SECRET:DYPYXVYCV5DBLOVTOKWZ42LEFFQ1TRZHK23UGG0TYJWDYKGU


In [16]:
#Function to get data from foursquare
def getNearbyVenues(names, latitudes, longitudes, radius=5000,categoryIds = '' ):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

<b>Let's call the previous method for the Toronto dataframe to get top 100 venues with a radius of 1kilometer</b>

In [17]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius
portuguese_restaurant_categoryId = '4def73e84765ae376e57713a' # https://developer.foursquare.com/docs/resources/categories

toronto_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude'],
                                 categoryIds=portuguese_restaurant_categoryId
                                  )

Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
The Beaches West,Indi

In [187]:
# Resulting dataframe size
print(toronto_venues.shape)

(1249, 7)


In [19]:
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Neighborhood'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [20]:
# Show venues in map
map_toronto_portuguese = folium.Map(location=[latitude, longitude], zoom_start=10)
addToMap(toronto_venues, 'red', map_toronto_portuguese)

map_toronto_portuguese

In [21]:
# Venues per Neighborhood
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",31,31,31,31,31,31
Agincourt,3,3,3,3,3,3
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",3,3,3,3,3,3
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",3,3,3,3,3,3
"Alderwood,Long Branch",4,4,4,4,4,4
"Bathurst Manor,Downsview North,Wilson Heights",1,1,1,1,1,1
"Bedford Park,Lawrence Manor East",3,3,3,3,3,3
Berczy Park,26,26,26,26,26,26
"Bloordale Gardens,Eringate,Markland Wood,Old Burnhamthorpe",3,3,3,3,3,3


#### Let's find out how many unique categories can be curated from all the returned venues

In [22]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 16 uniques categories.


In [23]:
toronto_venues['Venue Category'].unique()

array(['Asian Restaurant', 'Portuguese Restaurant', 'Bakery',
       'Dessert Shop', 'Pizza Place', 'Sandwich Place', 'Restaurant',
       'Gay Bar', 'Café', 'Seafood Restaurant',
       'Mediterranean Restaurant', 'Italian Restaurant', 'Beer Bar',
       'Wine Bar', 'Bar', 'BBQ Joint'], dtype=object)

### Analyze Each Neighborhood

In [24]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns[-1:]) + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(1249, 17)


,Neighborhood,Asian Restaurant,BBQ Joint,Bakery,Bar,Beer Bar,Café,Dessert Shop,Gay Bar,Italian Restaurant,Mediterranean Restaurant,Pizza Place,Portuguese Restaurant,Restaurant,Sandwich Place,Seafood Restaurant,Wine Bar
0,Woburn,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Cedarbrae,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,Cedarbrae,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Scarborough Village,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,"East Birchmount Park,Ionview,Kennedy Park",0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [25]:
toronto_onehot.shape

(1249, 17)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [26]:
toronto_grouped = toronto_onehot.groupby(["Neighborhood"]).mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped.head()

(90, 17)


,Neighborhood,Asian Restaurant,BBQ Joint,Bakery,Bar,Beer Bar,Café,Dessert Shop,Gay Bar,Italian Restaurant,Mediterranean Restaurant,Pizza Place,Portuguese Restaurant,Restaurant,Sandwich Place,Seafood Restaurant,Wine Bar
0,"Adelaide,King,Richmond",0.000000,0.0,0.290323,0.032258,0.032258,0.0,0.0,0.032258,0.032258,0.032258,0.0,0.419355,0.064516,0.032258,0.0,0.032258
1,Agincourt,0.333333,0.0,0.333333,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.333333,0.000000,0.000000,0.0,0.000000
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.333333,0.0,0.333333,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.333333,0.000000,0.000000,0.0,0.000000
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.0,0.000000
4,"Alderwood,Long Branch",0.000000,0.0,0.250000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.500000,0.250000,0.000000,0.0,0.000000


In [27]:
toronto_grouped.shape

(90, 17)

#### Get a frequency based data frame

First, let's write a function to sort the venues in descending order.

In [29]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [128]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Portuguese Restaurant,Bakery,Restaurant,Wine Bar,Sandwich Place,Mediterranean Restaurant,Italian Restaurant,Gay Bar,Beer Bar,Bar
1,Agincourt,Portuguese Restaurant,Bakery,Asian Restaurant,Wine Bar,Seafood Restaurant,Sandwich Place,Restaurant,Pizza Place,Mediterranean Restaurant,Italian Restaurant
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Portuguese Restaurant,Bakery,Asian Restaurant,Wine Bar,Seafood Restaurant,Sandwich Place,Restaurant,Pizza Place,Mediterranean Restaurant,Italian Restaurant
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Portuguese Restaurant,Wine Bar,Seafood Restaurant,Sandwich Place,Restaurant,Pizza Place,Mediterranean Restaurant,Italian Restaurant,Gay Bar,Dessert Shop
4,"Alderwood,Long Branch",Portuguese Restaurant,Restaurant,Bakery,Wine Bar,Seafood Restaurant,Sandwich Place,Pizza Place,Mediterranean Restaurant,Italian Restaurant,Gay Bar


### Cluster Neighborhoods using K-means

In [129]:
# Run K means with 5 clusters

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 3, 3, 2, 1, 2, 0, 1, 1, 1])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [130]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.merge(neighborhoods_venues_sorted, on="Neighborhood", how="right")

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1G,Scarborough,Woburn,43.770992,-79.216917,3,Asian Restaurant,Wine Bar,Seafood Restaurant,Sandwich Place,Restaurant,Portuguese Restaurant,Pizza Place,Mediterranean Restaurant,Italian Restaurant,Gay Bar
1,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,3,Portuguese Restaurant,Asian Restaurant,Wine Bar,Seafood Restaurant,Sandwich Place,Restaurant,Pizza Place,Mediterranean Restaurant,Italian Restaurant,Gay Bar
2,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,2,Portuguese Restaurant,Wine Bar,Seafood Restaurant,Sandwich Place,Restaurant,Pizza Place,Mediterranean Restaurant,Italian Restaurant,Gay Bar,Dessert Shop
3,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029,2,Portuguese Restaurant,Wine Bar,Seafood Restaurant,Sandwich Place,Restaurant,Pizza Place,Mediterranean Restaurant,Italian Restaurant,Gay Bar,Dessert Shop
4,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577,2,Portuguese Restaurant,Wine Bar,Seafood Restaurant,Sandwich Place,Restaurant,Pizza Place,Mediterranean Restaurant,Italian Restaurant,Gay Bar,Dessert Shop


In [132]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=1).add_to(map_clusters)
       
map_clusters

In [34]:
df1 = toronto_venues.iloc[:,3:6]
df1.drop_duplicates(inplace=True)
df1.shape

(67, 3)

### Cluster Venues using DBSCAN

In [188]:
# Clustering with DBScan

from sklearn.cluster import DBSCAN
import sklearn.utils
from sklearn.preprocessing import StandardScaler
sklearn.utils.check_random_state(1000)

Clus_dataSet = df1[['Venue Latitude','Venue Longitude']]
Clus_dataSet = np.nan_to_num(Clus_dataSet)
Clus_dataSet = StandardScaler().fit_transform(Clus_dataSet)

# Compute DBSCAN
db = DBSCAN(eps=0.5, min_samples=2).fit(Clus_dataSet)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_
df1["Clus_Db"]=labels

realClusterNum=len(set(labels)) - (1 if -1 in labels else 0)
clusterNum = len(set(labels)) 

In [106]:
# create map for venues
map_clustersDbScan = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(df1.Clus_Db.max() + 1)
ys = [i + x + (i*x)**2 for i in range(df1.Clus_Db.max() + 1)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df1['Venue Latitude'], df1['Venue Longitude'], df1['Venue'], df1['Clus_Db']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    
    selectedColor = rainbow[cluster-1]
    selectedRainbow = rainbow[cluster-1]
    if cluster == -1:
        selectedColor = 'k'
        selectedRainbow = 'k'
    
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=selectedColor,
        fill=True,
        fill_color=selectedRainbow,
        fill_opacity=1).add_to(map_clustersDbScan)
       
map_clustersDbScan

In [174]:
# Merge Venues with existing neighbourhoods
newDf = toronto_venues.iloc[:,0:6]
newDf.head()
newDf = newDf.merge(df1, left_on=["Venue","Venue Latitude","Venue Longitude"], right_on=["Venue","Venue Latitude","Venue Longitude"], how="left", validate="many_to_one")
newDf.shape

(1249, 7)

In [175]:
# create map for neighboors
map_clustersDbScan = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(newDf.Clus_Db.max() + 1)
ys = [i + x + (i*x)**2 for i in range(newDf.Clus_Db.max() + 1)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(newDf['Neighborhood Latitude'], newDf['Neighborhood Longitude'], newDf['Neighborhood'], newDf['Clus_Db']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    
    selectedColor = rainbow[cluster-1]
    selectedRainbow = rainbow[cluster-1]
    if cluster == -1:
        selectedColor = 'k'
        selectedRainbow = 'k'
    
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=selectedColor,
        fill=True,
        fill_color=selectedRainbow,
        fill_opacity=1).add_to(map_clustersDbScan)
       
map_clustersDbScan

In [39]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Woburn,43.770992,-79.216917,Boston Steak House 波士頓餐廳,43.787744,-79.267428,Asian Restaurant
1,Cedarbrae,43.773136,-79.239476,Nando's Flame-Grilled Chicken,43.773113,-79.281166,Portuguese Restaurant
2,Cedarbrae,43.773136,-79.239476,Boston Steak House 波士頓餐廳,43.787744,-79.267428,Asian Restaurant
3,Scarborough Village,43.744734,-79.239476,Nando's Flame-Grilled Chicken,43.773113,-79.281166,Portuguese Restaurant
4,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029,Portugril,43.725819,-79.312785,Portuguese Restaurant


### Analyze Clusters (K-means)

In [135]:
def getClusterValueCounts(clusterLabel):
    cluster = toronto_merged.loc[toronto_merged['Cluster Labels'] == clusterLabel, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
    
    venue_types = cluster.iloc[ : , 2]
    
    for x in range(3, 12) :
        venue_types = venue_types.append(cluster.iloc[ : , x])
    
    df = pd.DataFrame({'MostCommon' : cluster.iloc[ : , 2].value_counts()[0:5], 'All' : venue_types.value_counts()[0:5]})
    df.sort_values(by=['MostCommon'], inplace=True, ascending=False)
    return df


In [145]:
# Cluster 1
getClusterValueCounts(0)

,MostCommon,All
Portuguese Restaurant,9.0,14.0
BBQ Joint,1.0,NaN
Bakery,1.0,NaN
Sandwich Place,1.0,14.0
Seafood Restaurant,1.0,14.0
Restaurant,NaN,14.0
Wine Bar,NaN,14.0


In [177]:
# Cluster 2
getClusterValueCounts(1)

,MostCommon,All
Portuguese Restaurant,46.0,46
Bakery,NaN,44
Mediterranean Restaurant,NaN,43
Restaurant,NaN,46
Wine Bar,NaN,42


In [178]:
# Cluster 3
getClusterValueCounts(2)

,MostCommon,All
Portuguese Restaurant,24.0,NaN
Italian Restaurant,NaN,24.0
Pizza Place,NaN,24.0
Restaurant,NaN,24.0
Seafood Restaurant,NaN,24.0
Wine Bar,NaN,24.0


In [140]:
# Cluster 3
getClusterValueCounts(4)

,MostCommon,All


In [139]:
# Cluster 3
getClusterValueCounts(3)

,MostCommon,All
Portuguese Restaurant,5.0,NaN
Asian Restaurant,1.0,6.0
Italian Restaurant,NaN,6.0
Pizza Place,NaN,6.0
Restaurant,NaN,6.0
Seafood Restaurant,NaN,6.0


In [209]:
for x in range(0,4):
    print('\nCluster ' + str(x))
    print(toronto_merged.loc[toronto_merged['Cluster Labels'] == x]['Neighborhood'])


Cluster 0
31                                  Lawrence Park
32                               Davisville North
33                             North Toronto West
34                                     Davisville
49               Bedford Park,Lawrence Manor East
58                Lawrence Heights,Lawrence Manor
59                                      Glencairn
66               Downsview,North Park,Upwood Park
67    Del Ray,Keelesdale,Mount Dennis,Silverthorn
68                   The Junction North,Runnymede
77    The Kingsway,Montgomery Road,Old Mill North
80                               Islington Avenue
85                                         Weston
86                                      Westmount
Name: Neighborhood, dtype: object

Cluster 1
17                           CFB Toronto,Downsview East
19                                    Downsview Central
25                                              Leaside
26                                     Thorncliffe Park
27                  

### Analyze Clusters (DBSCAN)

In [203]:
for x in range(-1, 5):
    t = newDf[newDf['Clus_Db'] == x]
    print('\n')
    print('Cluster '+ str(x))
    print(t.Neighborhood.unique().tolist())



Cluster -1
['East Birchmount Park,Ionview,Kennedy Park', 'Clairlea,Golden Mile,Oakridge', 'Dorset Park,Scarborough Town Centre,Wexford Heights', 'Maryvale,Wexford', 'York Mills West', 'Parkwoods', 'Don Mills North', 'Flemingdon Park,Don Mills South', 'Bathurst Manor,Downsview North,Wilson Heights', 'Victoria Village', 'Woodbine Gardens,Parkview Hill', 'Woodbine Heights', 'The Beaches', 'Leaside', 'Thorncliffe Park', 'East Toronto', 'The Danforth West,Riverdale', 'The Beaches West,India Bazaar', 'Studio District', 'Lawrence Park', 'Davisville North', 'North Toronto West', 'Davisville', 'Moore Park,Summerhill East', 'Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West', 'Rosedale', 'Cabbagetown,St. James Town', 'Church and Wellesley', 'Harbourfront,Regent Park', 'Ryerson,Garden District', 'St. James Town', 'Berczy Park', 'Central Bay Street', 'Adelaide,King,Richmond', 'Design Exchange,Toronto Dominion Centre', 'Commerce Court,Victoria Hotel', 'Roselawn', 'Forest Hill North,Fo